# Transfer Learning Classifier with CNTK (no Keras)

We start with code provided by the [Transfer Learning example](https://github.com/Microsoft/CNTK/tree/master/Examples/Image/TransferLearning) in the [CNTK repo](https://github.com/Microsoft/CNTK). CNTK binaries are installed on a Data Science Virtual Machine, however not the sample code, which is what we've after.

In [1]:
!git clone https://github.com/Microsoft/CNTK.git

Cloning into 'CNTK'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 214632 (delta 33), reused 20 (delta 8), pack-reused 214561
Receiving objects: 100% (214632/214632), 906.19 MiB | 58.67 MiB/s, done.
Resolving deltas: 100% (156018/156018), done.
Checking connectivity... done.
Checking out files: 100% (3078/3078), done.


We add the path to the module that downloads models pretrained using CNTK and the path to the Transfer Learning example.

In [2]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), 'CNTK', 'PretrainedModels'))
sys.path.append(os.path.join(os.getcwd(), 'CNTK', 'Examples', 'Image', 'TransferLearning'))

In [3]:
import cntk, requests, shutil, subprocess
print (cntk.__version__)

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

from __future__ import print_function
from download_model import * # From CNTK repo
from io import BytesIO
from PIL import Image, ImageOps
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from TransferLearning import * # From CNTK repo

2.5.1


In [4]:
%matplotlib inline

In [5]:
%load_ext autoreload
%autoreload 2

We follow the example and download ResNet 18 trained with CNTK using ImageNet.

In [6]:
download_model_by_name("ResNet18_ImageNet_CNTK")

Saved model as /data/home/graeme/notebooks/openhack/CNTK/PretrainedModels/ResNet18_ImageNet_CNTK.model


Confirm you can see ResNet18_ImageNet_CNTK.model in ~/CNTK/PretrainedModels.

Here we define a function to create train and validate folders. The function splits the content of the source folder, preserving the subfolders of the source folder when copying images to the train folder, and copies the remainder directly to the validate folder.

Images are also resized to (224, 224) to match the input for the ResNet 18 model.

In [7]:
def create_train_validate_folders(source_folder_path, train_folder_path, validate_folder_path, size, validate_size=0.30):
    if (os.path.isdir(train_folder_path)) and (os.path.exists(train_folder_path)):
        shutil.rmtree(train_folder_path)
        
    if (os.path.isdir(validate_folder_path)) and (os.path.exists(validate_folder_path)):
        shutil.rmtree(validate_folder_path)
    
    os.makedirs(validate_folder_path)
    
    for dirpath, dirnames, filenames in os.walk(source_folder_path):
        for dirname in dirnames:
            files = os.listdir(os.path.join(dirpath, dirname))
            train, validate = train_test_split(files, test_size=validate_size)
            
            train_output_path = os.path.join(train_folder_path, dirname)
            
            os.makedirs(train_output_path)
            
            for file_name in train:
                src = os.path.join(dirpath, dirname, file_name)
                dst = os.path.join(train_output_path, '{0}.jpg'.format(file_name.split('.')[0]))
                image = Image.open(src)
                new_image = resize_image(image, size)
                new_image.save(dst)
            
            for file_name in validate:
                src = os.path.join(dirpath, dirname, file_name)
                dst = os.path.join(validate_folder_path, '{0}.jpg'.format(file_name.split('.')[0]))
                image = Image.open(src)
                new_image = resize_image(image, size)
                new_image.save(dst)
                
def resize_image(image, size):
    if np.array(image).shape[2] == 4:
        image = image.convert('RGB')
        
    image.thumbnail(size, Image.ANTIALIAS)
    new_image = Image.new("RGB", size, (255, 255, 255))
    new_image.paste(image, (int((size[0] - image.size[0]) / 2), int((size[1] - image.size[1]) / 2)))
    
    return new_image

These three functions are copied from ~/CNTK/Examples/Image/TransferLearning/TransferLearing_Extended.py to create the file that maps each image to its class.

In [8]:
def create_map_file_from_folder(root_folder, class_mapping, include_unknown=False):
    map_file_name = os.path.join(root_folder, "map.txt")
    lines = []
    for class_id in range(0, len(class_mapping)):
        folder = os.path.join(root_folder, class_mapping[class_id])
        if os.path.exists(folder):
            for entry in os.listdir(folder):
                filename = os.path.join(folder, entry)
                if os.path.isfile(filename) and os.path.splitext(filename)[1] in file_endings:
                    lines.append("{0}\t{1}\n".format(filename, class_id))

    if include_unknown:
        for entry in os.listdir(root_folder):
            filename = os.path.join(root_folder, entry)
            if os.path.isfile(filename) and os.path.splitext(filename)[1] in file_endings:
                lines.append("{0}\t-1\n".format(filename))

    lines.sort()
    with open(map_file_name , 'w') as map_file:
        for line in lines:
            map_file.write(line)

    return map_file_name

def create_class_mapping_from_folder(root_folder):
    classes = []
    for _, directories, _ in os.walk(root_folder):
        for directory in directories:
            classes.append(directory)
    classes.sort()
    return np.asarray(classes)

def format_output_line(img_name, true_class, probs, class_mapping, top_n=3):
    class_probs = np.column_stack((probs, class_mapping)).tolist()
    class_probs.sort(key=lambda x: float(x[0]), reverse=True)
    top_n = min(top_n, len(class_mapping)) if top_n > 0 else len(class_mapping)
    true_class_name = class_mapping[true_class] if true_class >= 0 else 'unknown'
    line = '[{"class": "%s", "predictions": {' % true_class_name
    for i in range(0, top_n):
        line = '%s"%s":%.3f, ' % (line, class_probs[i][1], float(class_probs[i][0]))
    line = '%s}, "image": "%s"}]\n' % (line[:-2], img_name.replace('\\', '/').rsplit('/', 1)[1])
    return line

We also copy the function to train and evaluate our new model, adding a statement to return the new model when done.

In [9]:
def train_and_eval(_base_model_file, _train_image_folder, _test_image_folder, _results_file, _new_model_file, num_epochs, testing = False):
    # check for model and data existence
    if not (os.path.exists(_base_model_file) and os.path.exists(_train_image_folder) and os.path.exists(_test_image_folder)):
        print("Please run 'python install_data_and_model.py' first to get the required data and model.")
        exit(0)

    # get class mapping and map files from train and test image folder
    class_mapping = create_class_mapping_from_folder(_train_image_folder)
    train_map_file = create_map_file_from_folder(_train_image_folder, class_mapping)
    test_map_file = create_map_file_from_folder(_test_image_folder, class_mapping, include_unknown=True)
    
    # train
    trained_model = train_model(_base_model_file,
                                feature_node_name,
                                last_hidden_node_name,
                                image_width,
                                image_height,
                                num_channels,
                                len(class_mapping),
                                train_map_file,
                                num_epochs=num_epochs,
                                freeze=True)

    if not testing:
        trained_model.save(_new_model_file)
        print("Stored trained model at %s" % _new_model_file)

    # evaluate test images
    with open(_results_file, 'w') as output_file:
        with open(test_map_file, "r") as input_file:
            for line in input_file:
                tokens = line.rstrip().split('\t')
                img_file = tokens[0]
                true_label = int(tokens[1])
                probs = eval_single_image(trained_model, img_file, image_width, image_height)
                formatted_line = format_output_line(img_file, true_label, probs, class_mapping)
                output_file.write(formatted_line)

    print("Done. Wrote output to %s" % _results_file)
    
    return trained_model

We want to be able to view the progress of training by seeing a plot of the training loss and validation loss and so change ~/CNTK/Examples/Image/TransferLearning/TransferLearning.py to use TensorBoard.

Open ~/CNTK/Examples/Image/TransferLearning/TransferLearning.py and make the following changes:

- Replace line 24 with `from cntk.logging import log_number_of_parameters, ProgressPrinter, TensorBoardProgressWriter`
- Replace line 120 with `    progress_writers = [ProgressPrinter(tag='Training', num_epochs=num_epochs), TensorBoardProgressWriter(freq=10, log_dir=os.path.join(os.getcwd(), 'log'), model=tl_model)]`, and
- Replace line 121 with `trainer = Trainer(tl_model, (ce, pe), learner, progress_writers)`

File > Save.

Also, create the folder used by TensorBoard.

In [10]:
log_path = os.path.join(os.getcwd(), 'log')

if not os.path.exists(log_path):
    os.makedirs(log_path)

Then, configure the training run and start training...

In [11]:
base_folder = os.getcwd()

# Model
base_model_file = os.path.join(base_folder, 'CNTK', 'PretrainedModels', 'ResNet18_ImageNet_CNTK.model')

# Data
source_folder_path = os.path.join(base_folder, 'resized_images')
train_folder_path = os.path.join(base_folder, 'train')
validate_folder_path = os.path.join(base_folder, 'validate')

# Output
results_file = os.path.join(base_folder, "Output", "predictions.txt")
new_model_file = os.path.join(base_folder, "Output", "TransferLearning.model")

image_height = 224
image_width = 224
num_channels = 3
num_epochs = 30

feature_node_name = "features"
last_hidden_node_name = "z.x"
file_endings = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

create_train_validate_folders(source_folder_path, train_folder_path, validate_folder_path, (image_width, image_height))

try_set_default_device(gpu(0))

model = train_and_eval(base_model_file, train_folder_path, validate_folder_path, results_file, new_model_file, num_epochs)

Training transfer learning model for 30 epochs (epoch_size = 1480).
Training 6156 parameters in 2 parameter tensors.
Learning rate per minibatch: 0.2
Momentum per minibatch: 0.9
Finished Epoch[1 of 30]: [Training] loss = 1.054200 * 1480, metric = 29.39% * 1480 9.010s (164.3 samples/s);
Finished Epoch[2 of 30]: [Training] loss = 0.134829 * 1480, metric = 4.66% * 1480 3.406s (434.5 samples/s);
Finished Epoch[3 of 30]: [Training] loss = 0.087793 * 1480, metric = 2.77% * 1480 3.413s (433.6 samples/s);
Finished Epoch[4 of 30]: [Training] loss = 0.067547 * 1480, metric = 2.43% * 1480 3.417s (433.1 samples/s);
Finished Epoch[5 of 30]: [Training] loss = 0.044241 * 1480, metric = 1.22% * 1480 3.418s (433.0 samples/s);
Finished Epoch[6 of 30]: [Training] loss = 0.049837 * 1480, metric = 0.95% * 1480 3.430s (431.5 samples/s);
Finished Epoch[7 of 30]: [Training] loss = 0.050199 * 1480, metric = 1.69% * 1480 3.430s (431.5 samples/s);
Finished Epoch[8 of 30]: [Training] loss = 0.074643 * 1480, metri

At this point we have successfully trained a new convolutional neural network model with an accuracy greater than 0.9, which is saved to ~/Output.

After training, open TensorBoard to view a plot of training loss and validation loss. If you are using a Data Science Virtual Machine, after starting TensorBoard, add the port used by TensorBoard to the Network Security Group of your virtual machine, https://docs.microsoft.com/en-us/azure/virtual-machines/windows/nsg-quickstart-portal and access using the Public IP Address of your virtual machines.

In [ ]:
!tensorboard --logdir 'log'

To continue, interrupt the kernel to stop TensorBoard.

For the last task we need to score the five images that are not included in the ***gear*** dataset.

In [ ]:
test_folder_path = os.path.join(base_folder, 'test')

if (os.path.isdir(test_folder_path)) and (os.path.exists(test_folder_path)):
    shutil.rmtree(test_folder_path)

os.makedirs(test_folder_path)

image_urls = []
image_urls.append(('http://images.the-house.com/giro-g10mx-mtgy-07.jpg', 7))
image_urls.append(('https://i.stack.imgur.com/HeliW.jpg', 0))
image_urls.append(('https://productimages.camping-gear-outlet.com/e5/62379.jpg', 11))
image_urls.append(('http://s7d1.scene7.com/is/image/MoosejawMB/MIKAJMKFMKCAPNABx1024698_zm?$product1000$', 2))
image_urls.append(('http://www.buffalosystems.co.uk/wp-content/uploads/2012/06/zoom_apline_jacket_dark_russet-2365x3286.jpg', 5))

size = (image_width, image_height)

fig = plt.figure(figsize=(12, 8))

# Get the images and show the predicted classes
for url_idx in range(len(image_urls)):
    response = requests.get(image_urls[url_idx][0])
    image = Image.open(BytesIO(response.content))
    
    image_name = image_urls[url_idx][0].split('/')[-1]

    if not '.jpg' in image_name:
        image_name = '{0}.jpg'.format(image_name)
        
    image_path = os.path.join(test_folder_path, image_name)
    
    new_image = resize_image(image, size)
    new_image.save(image_path)
    
    true_label = image_urls[url_idx][1]
    class_mapping = create_class_mapping_from_folder(train_folder_path)
    
    probs = eval_single_image(model, image_path, image_width, image_height)
    formatted_line = format_output_line(image_path, true_label, probs, class_mapping)
    
    a=fig.add_subplot(1, len(image_urls), url_idx + 1)
    image_plot = plt.imshow(image)
    a.set_title(formatted_line.split('{')[2].split(',')[0])

(__-){